Description 

As we plan to open a San Francisco restaurant and assume we will rent a place.  
I couldn't get information about renting all neighborhoods in this dataset.

We only need to analyze the current rent range for this project.

In [1]:
import pandas as pd
import numpy as np
url='https://raw.githubusercontent.com/skybullet1987/BATTLE1/master/dataset.csv'
readfile = pd.read_csv(url)
readfile.head()

,NeighborhoodName,City,State,CountyName,2019-1,2019-2,2019-3,2019-04,2019-05,2019-06
0,Mission District,San Francisco,CA,San Francisco,3300,3250,3295,3500,3495,3450
1,Downtown,San Francisco,CA,San Francisco,2900,3010,3100,3050,3105,3150
2,Pacific Heights,San Francisco,CA,San Francisco,3600,3620,3610,3705,3723,3751
3,Nob Hill,San Francisco,CA,San Francisco,2902,2803,2850,2841,2863,2796
4,South of Market,San Francisco,CA,San Francisco,3620,3810,3830,3796,3754,3750


I only planned to test the data I had collected.



To find out which neighborhood is good to open a new restaurant, I will use a formula. Before I came up with a formula, I was wondering what all of the attributes / factors we can consider because it is really unfair to compare data from restaurant 10 years old with restaurant 1 year old.

Like for example, the 10-year-old restaurant's checkin count will be more compared to a 1-year-old or 6-month-old restaurant. And I also found, after analyzing data, that the number of checkins in many restaurants is zero. I thought it would be really helpful to figure out the number of crowds visiting a specific neighborhood, but I avoided it because of data discrepancy.

Then I concluded that there would definitely be ratings for every restaurant. Even if 100 customers have visited a restaurant that is 1 year old, the rating will be 5 stars, and the same applies to a restaurant that is 10 years old.

Below is the formula for the solution.
finalScore = (rentScore)0.5+(ratingScore)0.5

Closer the finalScore to 1, better neighborhood choice for the customer to decide. I rented more weights than ratings. Let's see the significance of is RentScore and RatingScore.

RentScore can be calculated by (maxrentneighborhood)/(maxrentneighborhood-minrentneighborhood). RentScore value can be obtained from the above rental dataset for each neighborhood.

RatingScore can be calculated by (maxred-restored)/(maxred-minred). Using foursquare location, ratingScore can be obtained. The steps to obtain ratingScore are below.

To analyze and obtain more information about other restaurants, we need the neighborhood coordinates. By passing the NeighborhoodName value through geocoding, you can obtain co-ordinates or latitude & longitude. Let's import the package and find second neighborhood coordinates from the rental dataset above.


In [2]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [4]:
address = 'Downtown, San Francisco, CA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Latitude and longitude of '+address+' are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


Latitude and longitude of Downtown, San Francisco, CA are 37.7875138, -122.407159.


address = 'Downtown, San Francisco, CA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Latitude and longitude of '+address+' are {}, {}.'.format(latitude, longitude))

Now once we got the neighborhood's latitude and longitude, let's use Foursquare Location to get the Mission District Neighborhood's all restaurant details. 
The restaurant details can be retrieved using search endpoint. 

We only need Russian restaurant data for our project, and in search endpoint there is an attribute called category I d, 
i.e. foursquare has a defined categoryid for each category (like Russian or Ukranian or georgian restaurant) that will help us get the desired data.

In [5]:
CLIENT_ID = 'IZYIHS5CH11SQSDVLQ0SPHMZOJUAGKZWBSEDNLJP4CCEKH4W' # your Foursquare ID
CLIENT_SECRET = '0NI2OCB1CRFX02GUZ3ZLQEQJ3MSBMNN2NPVHM2T2ZFKAZG0L' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: IZYIHS5CH11SQSDVLQ0SPHMZOJUAGKZWBSEDNLJP4CCEKH4W
CLIENT_SECRET:0NI2OCB1CRFX02GUZ3ZLQEQJ3MSBMNN2NPVHM2T2ZFKAZG0L


In [6]:
categoryId= '5293a7563cf9994f4e043a44'   #categoryID for Russiann restaurant.
radius=1000                              # radius in meters

        



In [28]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT,
            categoryId)

In [29]:
#Send the GET request and examine the resutls
import requests 
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cedfe904c1f6753b3705a3f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown San Francisco-Union Square',
  'headerFullLocation': 'Downtown San Francisco-Union Square, San Francisco',
  'headerLocationGranularity': 'neighborhood',
  'query': 'russian',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 37.79651380900001,
    'lng': -122.39579200648618},
   'sw': {'lat': 37.77851379099999, 'lng': -122.41852599351381}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '58cc4d2fe0adac17bbf4838e',
       'name': 'Pushkin',
       'location': {'address': '380 Bush St',
        'crossStreet': 'at Kearny St',
        'lat': 37.7909430107134

Now we got all the details of the restaurants in Downtown that is in the neighborhood's 1000-meter radius in json format. 
Let's check in json's response how many restaurants are there.

In [30]:
Restaurants= len(results['response'])
print ('There are',  Restaurants , 'restaurants.')

There are 8 restaurants.


Let's also get one venue id from json response and use VenueID end point of foursquare to get venue details

In [31]:
url2 = 'https://api.foursquare.com/v2/venues/58cc4d2fe0adac17bbf4838e?&client_id={}&client_secret={}&v={}&ll={},{}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude
    )
results2 = requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '5cedff1b6a607149395b04bf'},
 'response': {'venue': {'id': '58cc4d2fe0adac17bbf4838e',
   'name': 'Pushkin',
   'contact': {'phone': '4159013434',
    'formattedPhone': '(415) 901-3434',
    'instagram': 'pushkinsf',
    'facebook': '392957711051136',
    'facebookUsername': 'pushkinsf',
    'facebookName': 'Pushkin'},
   'location': {'address': '380 Bush St',
    'crossStreet': 'at Kearny St',
    'lat': 37.79094301071348,
    'lng': -122.40387738626474,
    'labeledLatLngs': [{'label': 'display',
      'lat': 37.79094301071348,
      'lng': -122.40387738626474}],
    'distance': 478,
    'postalCode': '94104',
    'cc': 'US',
    'city': 'San Francisco',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['380 Bush St (at Kearny St)',
     'San Francisco, CA 94104',
     'United States']},
   'canonicalUrl': 'https://foursquare.com/v/pushkin/58cc4d2fe0adac17bbf4838e',
   'categories': [{'id': '5293a7563cf9994f4e043a44',
    

In [32]:
restaurantrating= results2['response']['venue']['rating']
restaurantrating

9.5

Like this, we can get the rating value from json for each neighborhood and calculate the final score.

* Please note that I showed an example in this data section for only one neighborhood. For all neighborhoods, I will use the loop to execute the formula and final results. I will do all the calculation part for all neighborhoods in the reporting methodology section.